# Hotel Recommendation System(MINI PROJECT)


# COD3

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import math
import os
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


In [33]:
hotel_details=pd.read_csv('/content/hoteldetails.csv',delimiter=',')
hotel_rooms=pd.read_csv('/content/hotelrooms.csv',delimiter=',')

In [35]:
hotel_details

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4.0,43.440124,16.682505,2.0,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3.0,53.012329,18.603800,5.0,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3.0,46.477920,11.813350,2.0,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3.0,37.452316,25.329849,5.0,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3.0,46.682780,11.131736,2.0,http://www.booking.com/hotel/it/pension-morenf...,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69709,116115,529015,Hotel Poseidon,Kamari - Santoni,Santorini,Greece,84700.0,Hotels,2.0,36.372322,25.482658,5.0,https://www.agoda.com/en-gb/hotel-poseidon/hot...,EUR
69710,116116,5290514,Residence Amarena,6 Rue du 19 Mars 1962,Saint-Paul-les-Dax,France,40990.0,Hotels,3.0,43.729790,-1.050540,5.0,https://www.agoda.com/en-gb/residence-amarena/...,EUR
69711,116117,5290860,Hotel Arianna,"Viale Abruzzi, 105",Cervia,Italy,48015.0,Hotels,2.0,44.223572,12.380760,5.0,https://www.agoda.com/en-gb/hotel-arianna/hote...,EUR
69712,116118,529114,Hotel Allegro Wien,"Matzleinsdorferplatz 1, A-1050 Vienna",Vienna,Austria,1050.0,Hotels,3.0,48.181015,16.358435,5.0,https://www.agoda.com/en-gb/hotel-allegro-wien...,EUR


In [36]:
hotel_details.head(10)

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4.0,43.440124,16.682505,2.0,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3.0,53.012329,18.603800,5.0,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3.0,46.477920,11.813350,2.0,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3.0,37.452316,25.329849,5.0,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3.0,46.682780,11.131736,2.0,http://www.booking.com/hotel/it/pension-morenf...,EUR
5,46411,1771733,Boutique Hotel Three Lilies,Narodni 3,Frantiskovy Lazne,Czech Republic,NaN,Hotels,4.0,50.118496,12.349790,2.0,http://www.booking.com/hotel/cz/three-lilies.e...,EUR
6,46412,177175,MPM Hotel Sport,"100, Pirin Str",Bansko,Bulgaria,2770.0,Hotels,4.0,41.824789,23.477644,2.0,http://www.booking.com/hotel/bg/sport.en-gb.html,BGN
7,46413,177176,Family Hotel Santo Bansko,6 St.Ivan Rilski Str,Bansko,Bulgaria,2770.0,Hotels,3.0,41.829603,23.481450,2.0,http://www.booking.com/hotel/bg/santo-bansko.e...,BGN
8,46414,177179,Villas Jagoda & Malina,Borovets,Borovets,Bulgaria,2010.0,Holiday parks,3.0,42.275284,23.618516,2.0,http://www.booking.com/hotel/bg/villa-settleme...,BGN
9,46415,177183,Hotel Silver,"10,Lozengrad",Sozopol,Bulgaria,8130.0,Hotels,3.0,42.417956,27.699371,2.0,http://www.booking.com/hotel/bg/silver-sozopol...,BGN


In [37]:
hotel_rooms.head()

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"


# Data Cleaning and transformations

In [38]:
del hotel_details['id']
del hotel_rooms['id']
del hotel_details['zipcode']

In [39]:
hotel_details=hotel_details.dropna()
hotel_rooms=hotel_rooms.dropna()

In [40]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [41]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

In [42]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription', 'hotelid',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'Source', 'url', 'curr'],
      dtype='object')

In [43]:
del hotel['hotelid']
del hotel['url']
del hotel['curr']
del hotel['Source']

In [44]:

hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude'],
      dtype='object')

# Recommender system based only on City and ratings about the hotel

In [45]:
def citybased(city):
    hotel['city']=hotel['city'].str.lower()
    citybase=hotel[hotel['city']==city.lower()]
    citybase=citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

In [46]:
print('Top 5 hotels')
citybased('Paris')

Top 5 hotels


,hotelname,starrating,address,roomamenities,ratedescription
12780,Hotel Mercure Paris Gobelins Place D'Italie,4.0,8 Bis Avenue De La Soeur Rosalie,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Room size: 28 m²/301 ft²
5292,Hotel Best Western Louvre Saint Honore,4.0,"141, rue Saint-Honore",Air conditioning: ;Coffee/tea maker: ;Daily ho...,Separate shower/bathtub
11127,Maxim Opera,4.0,"13, rue Geoffroy Marie",Air conditioning: ;Free Wi-Fi in all rooms!: ;...,"Pay nothing until November 29, 2019, Free Wi-F..."
11757,Best Western Paris CDG Airport Hotel,4.0,"1, Allee du Verger",Air conditioning: ;Alarm clock: ;Bathrobes: ;C...,"Room size: 20 m²/215 ft², Non-smoking, Bathtub..."
5426,Mercure Paris Roissy Charles de Gaulle Hotel,4.0,3 Allee du Verger,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 28 m²/301 ft², Non-smoking, Shower ..."


# Recommender system based only on Country and ratings about the hotel

In [47]:
def countrybased(country):
    hotel['country']=hotel['country'].str.lower()
    countrybase=hotel[hotel['country']==country.lower()]
    countrybase=countrybase.sort_values(by='starrating',ascending=False)
    countrybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(countrybase.empty==0):
        hname=countrybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

In [48]:
print('Top 5 hotels')
countrybased('Germany')

Top 5 hotels


,hotelname,starrating,address,roomamenities,ratedescription
620,Beach Hotel California,4.0,Deichweg 3,Air conditioning: ;Closet: ;Free bottled water...,"Room size: 12 m²/129 ft², Shower, 1 single bed"
3047,Hotel Mondial am Dom Cologne - MGallery,4.0,Kurt-Hackenberg-Platz 1,Air conditioning: ;Bathrobes: ;Blackout curtai...,"Room size: 16 m²/172 ft², City view, Non-smoki..."
5376,Courtyard Munich City East,4.0,Orleansstrasse 81-83,Air conditioning: ;Alarm clock: ;Blackout curt...,"Room size: 25 m²/269 ft², Courtyard view, Non-..."
4644,Das Reiners,4.0,Grub 20,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,"Free Breakfast, Pay nothing until October 26, ..."
11092,Hotel MODI,4.0,168 Schleissheimer Strasse,Air conditioning: ;Carpeting: ;Closet: ;Clothe...,"Room size: 24 m²/258 ft², Shower, 1 king bed"


In [49]:
room_no=[
     ('king',2),
   ('queen',2), 
    ('triple',3),
    ('master',3),
   ('family',4),
   ('murphy',2),
   ('quad',4),
   ('double-double',4),
   ('mini',2),
   ('studio',1),
    ('junior',2),
   ('apartment',4),
    ('double',2),
   ('twin',2),
   ('double-twin',4),
   ('single',1),
     ('diabled',1),
   ('accessible',1),
    ('suite',2),
    ('one',2)
   ]

In [50]:
def calc():
    guests_no=[]
    for i in range(hotel.shape[0]):
        temp=hotel['roomtype'][i].lower().split()
        flag=0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j]==room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag=1
                    break
            if flag==1:
                break
        if flag==0:
            guests_no.append(2)
    hotel['guests_no']=guests_no

calc()

In [51]:
hotel['roomamenities']=hotel['roomamenities'].str.replace(': ;',',')

In [52]:
def requirementbased(city,number,features):
    hotel['country']=hotel['country'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['country']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['city','hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)

In [53]:
import nltk
nltk.download('punkt')

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [55]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [56]:
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# sentence = "this is a foo bar sentence"}
# print [i for i in sentence.split() if i not in stop]

In [57]:
requirementbased('Germany',4,'Air conditioning')

,city,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription,similarity
0,schonberg (holstein),Beach Hotel California,Apartment,4,4.0,Deichweg 3,"air conditioning,carpeting,closet,coffee/tea m...","Room size: 25 m²/269 ft², Shower, Kitchenette,...",2
1,leipzig,Five Elements Hostel Leipzig,Family 4 People,4,2.0,"Kleine Fleischergasse, 8","air conditioning,blackout curtains,closet,clot...","Room size: 30 m²/323 ft², City view, Non-smoki...",2
24,frankfurt am main,Hotel Restaurant Cafe Schoene Aussicht,Apartment,4,3.0,24 Im Sperber,"air conditioning,closet,coffee/tea maker,dishw...","Room size: 100 m²/1076 ft², Garden view, Batht...",2
27,mindelheim,Hotel Kunstmuhle,Family Room,4,3.0,Hermelestrasse 19,"air conditioning,carpeting,closet,desk,hair dr...","Room size: 20 m²/215 ft², Balcony/terrace, Sho...",2
28,osann-monzel,Hotel Moselsteig,Apartment,4,3.0,Novianderweg 3,"air conditioning,alarm clock,closet,clothes ra...","Room size: 50 m²/538 ft², Balcony/terrace, Sho...",2
29,berchtesgaden,Hotel AlpinaRos Demming,Family,4,4.0,Sunklergasschen 2,"air conditioning,carpeting,closet,clothes rack...","Room size: 42 m²/452 ft², Mountain view, Balco...",2
33,kassel,Hotel Schweizer Hof,Apartment,4,3.0,Wilhelmshoher Allee 288,"air conditioning,daily newspaper,desk,free wi-...","Room size: 26 m²/280 ft², City view, Non-smoki...",2
35,oberasbach,Montana Hotel,Family Offer (2 Adults + 1 Child) - two separa...,4,3.0,Am Rathaus 5-7,"air conditioning,alarm clock,carpeting,closet,...","Room size: 15 m²/161 ft², Shower, 1 single bed...",2
39,papenburg,Hotel Walker am Stadtpark,Apartment - Ground Floor,4,3.0,Deverweg 27,"air conditioning,bathrobes,closet,clothes rack...","Room size: 40 m²/431 ft², Shower, Kitchenette,...",2
40,frankfurt am main,Tagungshotel Hochster Hof,Family Room,4,3.0,Mainberg 3-11,"air conditioning,closet,daily housekeeping,des...","Room size: 35 m²/377 ft², City view, Shower, 2...",2


Rate Description Based

In [58]:
def ratebased(city,number,features):
    hotel['country']=hotel['country'].str.lower()
    hotel['ratedescription']=hotel['ratedescription'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    rtbased=hotel[hotel['country']==city.lower()]
    rtbased=rtbased[rtbased['guests_no']==number]
    rtbased=rtbased.set_index(np.arange(rtbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    
    for i in range(rtbased.shape[0]):
        temp_tokens=word_tokenize(rtbased['ratedescription'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        
        cos.append(len(rvector))
    rtbased['similarity']=cos
    rtbased=rtbased.sort_values(by='similarity',ascending=False)
    rtbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return rtbased[['country','hotelname','roomtype','guests_no','starrating','address','ratedescription','similarity']].head(10)

In [59]:
ratebased('germany',2,'free wi-fi')

,country,hotelname,roomtype,guests_no,starrating,address,ratedescription,similarity
370,germany,Tagungshotel Hochster Hof,Double Room,2,3.0,Mainberg 3-11,"free wi-fi, extra low price! (non-refundable)",2
259,germany,Courtyard Munich City East,"Superior Room, Guest room, 2 Twin/Single Bed(s)",2,4.0,Orleansstrasse 81-83,"free wi-fi, extra low price! (non-refundable)",2
142,germany,Das Reiners,Double Superior,2,4.0,Grub 20,"free breakfast, pay nothing until october 24, ...",2
349,germany,Grand Palace Hotel Hannover,Twin room sole use,2,4.0,Lavesstrasse 77,"free breakfast, pay nothing until december 6, ...",2
286,germany,Montana Hotel,Quadruple Room,2,3.0,Am Rathaus 5-7,"pay at the hotel, free wi-fi, free cancellation",2
477,germany,Goethe Conference Messe Hotel,Suite,2,3.0,"Schmidtstrasse 57, Gallusviertel","free wi-fi, extra low price! (non-refundable)",2
436,germany,Parkhotel Eschweiler,Comfort Quadruple Room,2,3.0,Parkstr. 14-16,"free wi-fi, free cancellation before december ...",2
331,germany,Novotel Suites Berlin City Potsdamer Platz,Superior Suite with 2 Single Beds,2,3.0,Anhalter Strasse 2,"room size: 30 m²/323 ft², city view, non-smoki...",0
318,germany,Rheinhotel Wagner,Double Room,2,2.0,Burgenstrasse 6,"room size: 18 m²/194 ft², shower, 1 queen bed",0
316,germany,Hotel Kuchalber Hof,Business Double Room,2,3.0,Kuchalb 10,"room size: 25 m²/269 ft², shower, 1 queen bed",0
